In [1]:
import os
import random
import shutil  
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# train-test split
root_dir = 'cell_images'
class_dirs = ['Parasitized','Uninfected']

train_path = os.path.join('data','train')  
test_path = os.path.join('data','test')

test_ratio = 0.10

for class_dir in class_dirs:
    # create directories
    os.makedirs(os.path.join(train_path,class_dir))
    os.makedirs(os.path.join(test_path,class_dir))
    # shuffle images
    src = os.path.join(root_dir,class_dir) 
    file_list = os.listdir(src)
    random.shuffle(file_list)
    threshold = int(len(file_list) * (1 - test_ratio))
    train_files, test_files = file_list[0:threshold], file_list[threshold:]
    # append path to file names
    train_files = [os.path.join(src,name) for name in train_files]
    test_files = [os.path.join(src,name) for name in test_files]
    # Copy-pasting images
    for name in train_files:
        shutil.copy(name, os.path.join(train_path,class_dir))
    for name in test_files:
        shutil.copy(name, os.path.join(test_path,class_dir))
    
    print(f'Total {class_dir} images: ', len(file_list))
    print('Training: ', len(train_files))
    print('Testing: ', len(test_files))
    print()


Total Parasitized images:  13780
Training:  12402
Testing:  1378

Total Uninfected images:  13780
Training:  12402
Testing:  1378

